<h1 style="color:red;"> SQL Injection + </h1>
<span> Chou@ibCher+</span>


In [6]:
import pandas as pd
import os
import sys
import os

In [ ]:
# Correct the path
current_dir = os.getcwd()  
parent_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))
print('Current directory:', parent_dir)

# Define the file path
df_path = os.path.join(parent_dir, 'data', 'processed', 'cleanedData.csv')

# Load the dataset
df = pd.read_csv(df_path, usecols=['payload', 'is_malicious', 'injection_type'])


print('Data distribution')
print(df['injection_type'].value_counts())


Current directory: c:\Users\LENOVO\Documents\GitHub\WAF-AI
Data distribution
injection_type
LEGAL    15257
SQL       3288
Name: count, dtype: int64


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load the dataset

df = pd.read_csv('payloads.csv', usecols=['payload', 'is_malicious', 'injection_type'])

# Convert the 'payload' column to strings and fill NaN values
df['payload'] = df['payload'].astype(str).fillna('')

# Remove any empty data points
df = df[df['payload'] != '']

# Remove any duplicate payloads
df = df.drop_duplicates(subset=['payload'])

# Custom tokenization function to capture SQL injection patterns
def custom_tokenizer(text):
    tokens = text.split()
    return tokens

# Initialize the Count Vectorizer (Bag of Words) with custom tokenizer and n-grams
count_vectorizer = CountVectorizer(min_df=1, tokenizer=custom_tokenizer, ngram_range=(1, 3))

# Transform the 'payload' column
X = count_vectorizer.fit_transform(df['payload'])

# Define the target variable
y = df['is_malicious']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Naive Bayes classifier
nb_classifier = MultinomialNB()

# Train the classifier
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

# Save the model and vectorizer

modelPathSaving = os.path.join(parent_dir, 'WAF', 'nb.pkl')
vectorizerPathSaving = os.path.join(parent_dir, 'WAF', 'vectorizer.pkl')

joblib.dump(nb_classifier, modelPathSaving)
joblib.dump(count_vectorizer, vectorizerPathSaving)

FileNotFoundError: [Errno 2] No such file or directory: 'payloads.csv'

In [7]:
# SQL injection examples
sql_injections = [
    'verve',
    'helllo chouaib',
    'username',
    'password',
    'bounjour',
    "1' OR '1'='1",
    "1' OR '1'='1' --",
    "1' OR '1'='1' ({",
    "1' OR '1'='1' /*",
    "1' OR '1'='1' #",
    "1' OR '1'='1' AND '1'='1",
    "1' OR '1'='1' AND '1'='2",
    "1' OR '1'='1' UNION SELECT NULL, NULL",
    "1' OR '1'='1' UNION SELECT username, password FROM users",
    "1' OR '1'='1' UNION SELECT table_name, column_name FROM information_schema.columns"
]

# Transform the SQL injections using the vectorizer
sql_injections_vectorized = count_vectorizer.transform(sql_injections).toarray()

# Predict using the Naive Bayes model
predictions = nb_classifier.predict(sql_injections_vectorized)

# Print the predictions
for i, sql in enumerate(sql_injections):
    print(f"SQL Injection: {sql} -> Prediction: {predictions[i]}")

SQL Injection: verve -> Prediction: 0
SQL Injection: helllo chouaib -> Prediction: 0
SQL Injection: username -> Prediction: 0
SQL Injection: password -> Prediction: 0
SQL Injection: bounjour -> Prediction: 0
SQL Injection: 1' OR '1'='1 -> Prediction: 1
SQL Injection: 1' OR '1'='1' -- -> Prediction: 1
SQL Injection: 1' OR '1'='1' ({ -> Prediction: 1
SQL Injection: 1' OR '1'='1' /* -> Prediction: 1
SQL Injection: 1' OR '1'='1' # -> Prediction: 1
SQL Injection: 1' OR '1'='1' AND '1'='1 -> Prediction: 1
SQL Injection: 1' OR '1'='1' AND '1'='2 -> Prediction: 1
SQL Injection: 1' OR '1'='1' UNION SELECT NULL, NULL -> Prediction: 1
SQL Injection: 1' OR '1'='1' UNION SELECT username, password FROM users -> Prediction: 1
SQL Injection: 1' OR '1'='1' UNION SELECT table_name, column_name FROM information_schema.columns -> Prediction: 1
